# Screening tool for employers

Hello, Employer are you having problems getting through all the candidates that are applying for your graduate positions?
Well we have the solution for You!
This product will allow you to score personal statements before getting to the interview stage to save time on meeting applicants that do not meet the minimum requirements. 

## Preamble

In [1]:
import os
import numpy as np
import tritonclient.http as httpclient
import riva.client
import wave
import IPython

from tritonclient.utils import np_to_triton_dtype

auth = riva.client.Auth(uri='10.100.182.246:50051')
tts_service = riva.client.SpeechSynthesisService(auth)

URL = "10.98.96.143:8000"
MODEl_GPTJ_FASTERTRANSFORMER = "ensemble" 

OUTPUT_LEN = 128
BATCH_SIZE = 1
BEAM_WIDTH = 2
TOP_K = 1
TOP_P = 0.0

start_id = 220
end_id = 50256

client = httpclient.InferenceServerClient(URL,
                                           concurrency=1,
                                           verbose=False)

## Personal statement summary tools 
These functions will summararise the personal statement

In [2]:
# Inference hyperparameters
def prepare_tensor(name, input):
    tensor = httpclient.InferInput(
        name, input.shape, np_to_triton_dtype(input.dtype))
    tensor.set_data_from_numpy(input)
    return tensor

# explanation
def prepare_inputs(input0):
    bad_words_list = np.array([[""]], dtype=object)
    stop_words_list = np.array([[""]], dtype=object)
    input0_data = np.array(input0).astype(object)
    output0_len = np.ones_like(input0).astype(np.uint32) * OUTPUT_LEN
    runtime_top_k = (TOP_K * np.ones([input0_data.shape[0], 1])).astype(np.uint32)
    runtime_top_p = TOP_P * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    beam_search_diversity_rate = 0.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    temperature = 1.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    len_penalty = 1.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    repetition_penalty = 1.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    random_seed = 0 * np.ones([input0_data.shape[0], 1]).astype(np.int32)
    is_return_log_probs = True * np.ones([input0_data.shape[0], 1]).astype(bool)
    beam_width = (BEAM_WIDTH * np.ones([input0_data.shape[0], 1])).astype(np.uint32)
    start_ids = start_id * np.ones([input0_data.shape[0], 1]).astype(np.uint32)
    end_ids = end_id * np.ones([input0_data.shape[0], 1]).astype(np.uint32)

    inputs = [
        prepare_tensor("INPUT_0", input0_data),
        prepare_tensor("INPUT_1", output0_len),
        prepare_tensor("INPUT_2", bad_words_list),
        prepare_tensor("INPUT_3", stop_words_list),
        prepare_tensor("runtime_top_k", runtime_top_k),
        prepare_tensor("runtime_top_p", runtime_top_p),
        prepare_tensor("beam_search_diversity_rate", beam_search_diversity_rate),
        prepare_tensor("temperature", temperature),
        prepare_tensor("len_penalty", len_penalty),
        prepare_tensor("repetition_penalty", repetition_penalty),
        prepare_tensor("random_seed", random_seed),
        prepare_tensor("is_return_log_probs", is_return_log_probs),
        prepare_tensor("beam_width", beam_width),
        prepare_tensor("start_id", start_ids),
        prepare_tensor("end_id", end_ids),
    ]
    return inputs
good_words = ['computer']
bad_words = ['science']

#long input is a list of paragraphs
def statement_scoring(long_input, good_words, bad_words):
    score = 0
    for paragraph in longer_inputs:
        words = paragraph.lower().split(" ")
        for i in words:
            if i in good_words:
                score += 1
            if i in bad_words:
                score -= 1
    return score
def convert(s):
    # initialization of string to ""
    new = ""
    # traverse in the string
    for x in s:
        new += x
    # return string
    return [new]

def summerisation_preporcessing(personal_statement_path):
    f = open(personal_statement_path, "r")
    longer_inputs = []
    for x in f:
        longer_inputs.append(x)
    temp = [""]
    for i in longer_inputs:
        if i != '\n':
            temp += i
    temp2 = convert(temp)
    temp2.append('\n\ntl;dr:\n')
    return longer_inputs, temp2

def summary(text_to_summerise):
    input0 = [[text_to_summerise[0] + text_to_summerise[1]],]
    inputs = prepare_inputs(input0)
    result = client.infer(MODEl_GPTJ_FASTERTRANSFORMER, inputs)
    output0 = result.as_numpy("OUTPUT_0")
    return output0[0].decode('UTF-8').replace('<|endoftext|>', ' ')

## Personal statement rating tools
We will rate the personal statement using key words and a profanity filter.

In [3]:
#long input is a list of paragraphs
def statement_scoring(long_input, good_words, bad_words):
    score = 0
    for paragraph in longer_inputs:
        words = paragraph.lower().split(" ")
        for i in words:
            if i in good_words:
                score += 1
            if i in bad_words:
                score -= 1
    return score


### Full personal statement analysis


In [4]:
def complete_summerisation(personal_statement_path, good_words, bad_words):
    longer_inputs, temp2 = summerisation_preporcessing(personal_statement_path)
    summary_text = summary(temp2)
    score = statement_scoring(longer_inputs, good_words, bad_words)
    return score, summary_text

## Interview Functions

### Audio transcription
These functions are used to transcribe audio data from the interview

In [5]:
def scribe_audio(my_wav_file):
    asr_service = riva.client.ASRService(auth)

    offline_config = riva.client.RecognitionConfig(
        encoding=riva.client.AudioEncoding.LINEAR_PCM,
        max_alternatives=1,
        enable_automatic_punctuation=True,
        verbatim_transcripts=False,
    )
    
    riva.client.add_audio_file_specs_to_config(offline_config, my_wav_file)
    
    # BOOSTING
    # Add words that aren't scribed properly
    boosted_lm_words = ['word1','hello','name','UAV','CEZ','CDNN']
    boosted_lm_score = 20.0
    riva.client.add_word_boosting_to_config(offline_config, boosted_lm_words, boosted_lm_score)
    riva.client.add_audio_file_specs_to_config(offline_config, my_wav_file)
    
    with open(my_wav_file, 'rb') as fh:
        data = fh.read()
    
    response = asr_service.offline_recognize(data, offline_config)
    response = '%s' %response
    k =  response.find('transcript')
    m = len("transcript:")+2
    k2 = response.find('\n',k)
    string_response = response[k+m:k2-1]
    return string_response

# Main function

In [6]:
def summarise_statement_and_interview(path_to_statement, path_to_audio_dir, name):
#     Takes the path to a directory containing all appropriate audio files
#     Take path to personal statement
#     Returns a file containing a summarisation of the personal statement and the audio file
#     filename will be the name of the inverviewe and the score of their personal statement
    # Rate and summarise personal statement, note: the good_word and bad_words are just a 
    # stand-in for more complex ratings
    good_words = ['computer']
    bad_words = ['pay raise']
    statement_score, statement_summary = complete_summerisation(path_to_statement, good_words, bad_words)    
    # transcribe inverview
    interview_questions = ["1. Hello, Please describe yousrelf in 4 or less sentences",
                           "2. What is your previous work experience",
                           "3. How will you contribute to our team",
                           "4. Where do you see yourself in 5 years",
                           "5. What do you do on the weekend?",
                           "6. Do you have any questions for us?"]
    interview_answers = []
    file_names = os.listdir(path_to_audio_dir)
    for file_name in file_names:
        file_path = os.path.join(path_to_audio_dir, file_name)
        interview_answers.append(scribe_audio(file_path))
    
    # Add all data to output file
    with open(f"{name}_{statement_score}_EN.txt", "r") as f:
        f.write("PERSONAL STATEMENT SUMMARY\n\n")
        f.write(statement_summary)
        f.write("\n\n")
        f.write("INTERVIEW TRANSCRIPT\n\n")
        for id in range(len(interview_answers)):
            f.write(interview_questions[id])
            f.write("\n\t")
            f.write(interview_answers[id])
            f.write("\n\n")
        
            
        
        

In [7]:
summarise_statement_and_interview(r"./personal_statement.txt", r"./answer_audio", "Bob Ross") 

# Note: OUR AUDIO INTERFACE
We created a method for our user to record their interview question answers locally and then submit to our interviewing tool. This python script give susers a specified amount of time to answer and then stores it locally. The next step of this is to run a bash script which